### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [44]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [3]:
# Investigating the missing values
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


For the follow up action the values in target columns are in the format that must be changed.

In [4]:
#converting the target columns values.
cc_apps[13] = cc_apps[13].map({'+':1 , '-':0})

It seems there is no missing value. Taking a deep look in columns, shows the missing values mentioned as "?".

In [5]:
# Replacing the question marks with Nan 
cc_apps.replace('?',np.nan, inplace= True)
cc_apps.isna().sum().sort_values(ascending= False)

0     12
1     12
5      9
6      9
3      6
4      6
2      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
dtype: int64

As we saw in the dataset, some of the columns in numeric are in object. These such a columns must be changed to numeric.

In [6]:
# converting the numeric columns that are in object format.

col_to_convert = [1, 2, 7, 12, 13]
cc_apps[col_to_convert] = cc_apps[col_to_convert].apply(pd.to_numeric, errors = 'coerce')
cc_apps.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    float64
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    int64  
dtypes: float64(3), int64(3), object(8)
memory usage: 75.6+ KB


Now we need to handling the missing values. Although the number of missing values in the dataset are less than %5 and we can drop all of them, we prefere to impute them.

In [7]:
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,1


In [8]:
cc_apps.isna().sum().sort_values()

2      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
3      6
4      6
5      9
6      9
0     12
1     12
dtype: int64

I this step we have to divive the data set into train and test sections. before that we need to categories the data into categorical and numerical data sets. Then we can handling the missing data by applying the imputation from sklearn. Note that for the test part it must be just transform (not fit transform).

In [30]:
# divide the data set into train and test
X_cat = cc_apps.drop([1, 2, 7, 10, 12, 13], axis=1)
X_num = cc_apps.drop([0, 3, 4, 5, 6, 8, 9, 10, 11, 13], axis=1)
Y = cc_apps[13]
x_cat_train , x_cat_test, y_train, y_test = train_test_split(X_cat, Y, test_size = 0.2, random_state = 100)
x_num_train, x_num_test, y_train, y_test = train_test_split(X_num, Y, test_size = 0.2, random_state= 100)


In [31]:
# missing values imputation for categorical and numeric data seperately
imp_cat = SimpleImputer (strategy = "most_frequent")
imp_num = SimpleImputer()

x_cat_train = imp_cat.fit_transform(x_cat_train)
x_cat_test = imp_cat.transform(x_cat_test)

x_num_train = imp_num.fit_transform(x_num_train)
x_num_test = imp_num.transform(x_num_test)

# turning the categorical columns to dummy varaibles using OneHotEncoder
encoder = OneHotEncoder(sparse_output = False)
encoder.fit(x_cat_train)
x_cat_train_encoded = encoder.transform(x_cat_train)
x_cat_test_encoded = encoder.transform(x_cat_test)


In [38]:
# concatinating the dumy columns with the numeric culumns
x_train = np.append(x_cat_train_encoded , x_num_train, axis = 1)
x_test = np.append(x_cat_test_encoded, x_num_test, axis = 1)

In [40]:
# check the shape
print (x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(552, 42)
(138, 42)
(552,)
(138,)


In [48]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)
print(classification_report(y_test, y_pred))
print ("====================================")
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.76      0.83        78
           1       0.74      0.90      0.81        60

    accuracy                           0.82       138
   macro avg       0.82      0.83      0.82       138
weighted avg       0.83      0.82      0.82       138

[[59 19]
 [ 6 54]]


C:\Users\behza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
